## Import libraries

In [5]:
import sys
import numpy as np
import pandas as pd
import hypertools as hyp
from os.path import abspath, join as opj
from scipy.stats import pearsonr, sem
from scipy.interpolate import interp1d

%matplotlib inline

## Import analysis helpers

In [6]:
sys.path.insert(0, abspath('../../helpers/'))
from analysis_helpers import (
    N_TOPICS,
    VIDEO_WSIZE,
    RECALL_WSIZE,
    VECTORIZER_PARAMS,
    SEMANTIC_PARAMS,
    format_text,
    parse_windows,
    get_video_timepoints
)

Functions and variables used across multiple notebooks can be found [here](https://github.com/contextlab/sherlock-topic-model-paper/blob/master/code/helpers/analysis_helpers.py)

## Set paths & parameters

In [2]:
rawdir = '../../../data/raw/'
datadir = '../../../data/processed/'

## Define some functions

In [5]:
# wrap full topic modeling pipeline
def transform_video(annotations):
    scenes_list = annotations.apply(format_text, axis=1).tolist()
    video_windows, window_bounds = parse_windows(scenes_list, VIDEO_WSIZE)
    
    video_model = hyp.tools.format_data(video_windows, 
                                    vectorizer=VECTORIZER_PARAMS, 
                                    semantic=SEMANTIC_PARAMS, 
                                    corpus=video_windows)[0]
    
    tr_spans = video_text[['Start Time (TRs, 1.5s)', 'End Time (TRs, 1.5s)']]
    starts, stops = tr_spans.values.T
    video_model_TRs = np.empty((1976, 100))
    xvals = get_video_timepoints(window_bounds)
    xvals_TR = np.array(xvals) * 1976 / 2963
    TR_times = np.arange(1, 1977)
    interp_func = interp1d(xvals_TR, video_model, axis=0, fill_value='extrapolate')
    video_model_TRs = interp_func(TR_times)
    return video_model_TRs, video_windows

In [6]:
def interpolate_recall(recall_traj):
    n_windows = recall_traj.shape[0]
    window_xvals = np.linspace(0, 1976, n_windows)
    TR_times = np.arange(1, 1977)
    interp_func = interp1d(window_xvals, recall_traj, axis=0, fill_value='extrapolate')
    recall_interp = interp_func(TR_times)
    return recall_interp

In [7]:
def transform_recalls(recall_windows, video_windows):
    recall_models = hyp.tools.format_data(recall_windows, 
                                          vectorizer=VECTORIZER_PARAMS, 
                                          semantic=SEMANTIC_PARAMS, 
                                          corpus=video_windows)
    # need to interpolate recall trajectores to video trajectory 
    # length in order to correlate structure by timepoint
    recalls_interp = [interpolate_recall(r) for r in recall_models]
    return recalls_interp

In [8]:
def correlate_structures(traj1, traj2):
    corrmat1 = np.corrcoef(traj1)
    corrmat2 = np.corrcoef(traj2)
    triu1 = corrmat1[np.triu_indices_from(corrmat1, k=1)]
    triu2 = corrmat2[np.triu_indices_from(corrmat2, k=1)]
    return pearsonr(triu1, triu2)[0]

## Load & format data

In [9]:
video_text = pd.read_excel(opj(rawdir, 'Sherlock_Segments_1000_NN_2017.xlsx'))
video_text['Scene Segments'].fillna(method='ffill', inplace=True)
video_text.drop(index=[480, 481], inplace=True)
video_text.reset_index(drop=True, inplace=True)
video_text.loc[480:, 'Start Time (s) ': 'End Time (s) '] += video_text.loc[479, 'End Time (s) ']
keep_cols = np.append(video_text.columns[1:5], video_text.columns[6:15])
video_text = video_text.loc[:, keep_cols]
video_text.columns = list(video_text.columns[:4]) + ['Narrative details', 'Indoor vs outdoor', 
                                                     'Characters on screen', 'Character in focus', 
                                                     'Character speaking', 'Location', 'Camera angle', 
                                                     'Music presence', 'Text on screen']

# trajectories created from all features
full_trajs = np.load(opj(datadir, 'models_t100_v50_r10.npy'), allow_pickle=True)
full_video, full_recalls = full_trajs

In [10]:
recall_w = []
for sub in range(1, 18):
    transcript_path = opj(rawdir, f'NN{sub} transcript.txt')
    with open(transcript_path, 'r', encoding='cp1252') as f:
        recall = f.read().replace(b'\x92'.decode('cp1252'), "'").strip()
    recall_fmt = format_text(recall).split('.')
    if not recall_fmt[-1]:
        recall_fmt = recall_fmt[:-1]
    sub_recall_w = parse_windows(recall_fmt, RECALL_WSIZE)[0]
    recall_w.append(sub_recall_w)

## Iteratively hold out one feature and transform remaining

In [11]:
features = video_text.columns[4:]
# dropfeat_corrs = dict.fromkeys(features)
analyses = ['full vid corr', 'vid rec corr', 'vid rec sem']
dropfeat_corrs = pd.DataFrame(index=features, columns=analyses)

for feature in features:
    print(f'{feature}:')
    # transform remaining annotations
    partial_df = video_text.drop(feature, axis=1)
    other_features = partial_df.loc[:, partial_df.columns[4:]]
    dropfeat_traj, dropfeat_windows = transform_video(other_features)
    
    # compute similarity with full-feature video trajectory structure
    full_video_corr = correlate_structures(dropfeat_traj, full_video)
    
    # transform recalls using feature-removed corpus
    recall_trajs = transform_recalls(recall_w, dropfeat_windows)
    
    # compare structures to partial video model
    rec_corrs = np.array([correlate_structures(r, dropfeat_traj) 
                          for r in recall_trajs])
    feat_corr, feat_sem = rec_corrs.mean(), sem(rec_corrs)

    dropfeat_corrs.loc[feature] = [full_video_corr, feat_corr, feat_sem]
    print(f'\tsimilarity to full video: {full_video_corr}')
    print(f'\tvideo-recall structure similarity: {feat_corr}, SEM: {feat_sem}\n')
    
# add data for full model
rec_corr_full = np.array([correlate_structures(interpolate_recall(r), full_video) 
                          for r in full_recalls])
dropfeat_corrs.loc['All features'] = [1, rec_corr_full.mean(), sem(rec_corr_full)]
print('All features')
print(f'\tvideo-recall structure similarity: {rec_corr_full.mean()}, SEM: {sem(rec_corr_full)}')

Narrative details:
	similarity to full video: 0.8427530972814554
	video-recall structure similarity: 0.47881343463084924, SEM: 0.0169736012057851

Indoor vs outdoor:
	similarity to full video: 0.8959913399253916
	video-recall structure similarity: 0.645185590517598, SEM: 0.008412575567573256

Characters on screen:
	similarity to full video: 0.8534465690888688
	video-recall structure similarity: 0.5982795896546166, SEM: 0.010783902667291978

Character in focus:
	similarity to full video: 0.903622118494242
	video-recall structure similarity: 0.6340881813370098, SEM: 0.0086163187657276

Character speaking:
	similarity to full video: 0.9020093499625522
	video-recall structure similarity: 0.6639561160690202, SEM: 0.006833808429800328

Location:
	similarity to full video: 0.7814560702226314
	video-recall structure similarity: 0.5315939231688214, SEM: 0.012587585195835284

Camera angle:
	similarity to full video: 0.8744348125095095
	video-recall structure similarity: 0.5782115732347987, SEM: 

In [12]:
dropfeat_corrs.to_pickle(opj(datadir, 'feature_contribution.p'))